## Ex 3

In [2]:
from logger import FileProcessLogger, file_line_count

In [3]:
def get_words_and_tags():
    tag_of_word = {}
    words_with_tag = {}
    filename = './dane/supertags.txt'
    with open(filename, 'r', encoding='utf-8') as file:
        logger = FileProcessLogger(filename)
        for line in file:
            logger.update()
            word, tag = line.lower().rstrip('\n').split()
            tag_of_word[word] = tag
            if tag in words_with_tag:
                words_with_tag[tag].append(word)
            else:
                words_with_tag[tag] = [word]
    return tag_of_word, words_with_tag

In [4]:
tag_of_word, words_with_tag = get_words_and_tags()

Processed 10% of the file contents...
Processed 20% of the file contents...
Processed 30% of the file contents...
Processed 40% of the file contents...
Processed 50% of the file contents...
Processed 60% of the file contents...
Processed 70% of the file contents...
Processed 80% of the file contents...
Processed 90% of the file contents...
Processed 100% of the file contents...


In [5]:
def get_bigram():
    filename = './dane/poleval_2grams.txt'
    bigram = {}
    with open(filename, 'r', encoding='utf-8') as file:
        logger = FileProcessLogger(filename)
        for line in file:
            logger.update()
            n, word1, word2 = line.lower().split()
            n = int(n)
            if n < 10: 
                continue
            if word1 in bigram:
                bigram[word1].append((word2, n))
            else:
                bigram[word1] = [(word2, n)]
    return bigram

In [6]:
bigram = get_bigram()

Processed 10% of the file contents...
Processed 20% of the file contents...
Processed 30% of the file contents...
Processed 40% of the file contents...
Processed 50% of the file contents...
Processed 60% of the file contents...
Processed 70% of the file contents...
Processed 80% of the file contents...
Processed 90% of the file contents...
Processed 100% of the file contents...


In [7]:
from collections import defaultdict
def to_unigram(bigram):
    unigram = {}
    unigram = defaultdict(lambda: 0, unigram)
    for word1 in bigram:
        for word2, n in bigram[word1]:
            unigram[word1] += n
            unigram[word2] += n
    return unigram

In [8]:
unigram = to_unigram(bigram)

In [9]:
import numpy as np


def get_unigram_probability(word):
    if word in unigram:
        return unigram[word]
    return 0.0001


def choose_from_unigram(words):
    probs = np.array([get_unigram_probability(x) for x in words])
    probs = probs / np.sum(probs)
    return str(np.random.choice(words, 1, p=probs)[0])


def get_tag(word):
    if word in tag_of_word:
        return tag_of_word[word]
    return tag_of_word[('^' + word)[-3:]]


def random_similar_sentence(original):
    words = original.lower().split()
    tags = list(map(get_tag, words))
    alternative_words = list(map(lambda tag: words_with_tag[tag], tags))
    chosen_words = list(map(choose_from_unigram, alternative_words))
    return ' '.join(chosen_words)

In [10]:
sentences = [
    "Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę",
    "Litwo Ojczyzno moja Ty jesteś jak zdrowie",
    "Jeden z pojmanych najemników probował odebrać to ostrze",
    "Jak może być inaczej dopóki gildia znajduje się poza imperialną kontrolą",
    "Jak sobie życzysz",
    "Książę popatrzył spode łba i zajął się sterami",
]

In [11]:
for sentence in sentences:
    print(random_similar_sentence(sentence))

początkowy szymon podjął po niedalekiej zmianie niekiedy daną dolinę
sargo grado która ty żyjesz jak wykonywanie
jeden z zlokalizowanych rolników pkt nacisnąć to potomstwo
jak może być ponadto póki mowa porusza się poza społeczną wielkością
jak sobie chcesz
książę pojawił spode ratusza i opublikował się włoskami


## Ex 4

In [12]:
def get_bigram_probability(word, successor):
    if successor in bigram[word]:
        return bigram[word][successor]
    return 0.0001


def choose_from_bigram(word, successors):
    probs = np.array([get_bigram_probability(word, successor) for successor in successors])
    probs = probs / np.sum(probs)
    return str(np.random.choice(successors, 1, p=probs)[0])


def random_similar_sentence_v2(original):
    words = original.lower().split()
    tags = list(map(get_tag, words))
    alternative_words = list(map(lambda tag: words_with_tag[tag], tags))
    words_in_bigram = { word for word in bigram }
    
    best = ([], len(words))
    for _ in range(100):
        fails = 0
        known_alternatives = set(alternative_words[0])
        available_words = known_alternatives.intersection(words_in_bigram)
        word = choose_from_unigram(list(available_words))
        sentence = [word]
        for i in range(1, len(words)):
            known_alternatives = set(alternative_words[i])
            allowed_words = { successor for successor in bigram[word] }
            available_words = known_alternatives.intersection(allowed_words)

            if len(available_words) > 0:
                word = choose_from_bigram(word, list(available_words))
                sentence.append(word)
            else: 
                fails += 1      
                sentence.append('|')                
                allowed_words = { word for word in bigram }
                available_words = known_alternatives.intersection(allowed_words)
                word = choose_from_unigram(list(available_words))
                sentence.append(word)
        if fails < best[1]:
            best = (sentence, fails)
        
    return ' '.join(best[0])

In [13]:
for sentence in sentences:
    print(random_similar_sentence_v2(sentence))

ekonomiczny | farmer | zagrał | w | niedalekiej | stronie | tuż | wyrażoną | pieczę


KeyboardInterrupt: 

In [63]:
s = 'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę'
random_similar_sentence_v2(s)

'mazowiecki kombinator wyprzedził pomiędzy niesamodzielnej eurolidze kameralnie ukształtowaną zagadkę'

In [82]:
for _ in range(30):
    res = random_similar_sentence_v2(s)
    print(res)
    original_words = s.lower().split()
    res_words = res.split()
    for i in range(len(original_words)):
        wo = original_words[i]
        if wo not in tag_of_word:
            wo = wo[-3:]
        wr = res_words[i]
        if wr not in tag_of_word:
            wr = wr[-3:]
        to = tag_of_word[wo]
        tr = tag_of_word[wr]
        if to != tr:
            print(original_words[i], to, res_words[i], tr)

wojskowy | budnik | ukończył | w | niepodległej | polsce | poniżej | używaną | stronę


KeyError: '|'

In [81]:
for _ in range(1):
    res = random_similar_sentence_v2(s)
    print(res)
    res_words = res.split()
    for i in range(len(res_words)-1):
        w1, w2 = res_words[i:i+2]
        if w1 in bigram:
            if w2 not in bigram[w1]:
                print('failed for', w1, w2)
        if w1 not in bigram:
            alt = ('^'+w1)[-3:]
            if alt in bigram:
                if w2 not in bigram[alt][w2]:
                    print('failed for', alt, w2)
            else:
                print('no alt for', w1)

widoczny | pan | posiadł | w | niezwłocznej | formie | bardzo | uzgodnioną | olszynkę
failed for widoczny |
no alt for |
failed for pan |
no alt for |
failed for posiadł |
no alt for |
failed for w |
no alt for |
failed for niezwłocznej |
no alt for |
failed for formie |
no alt for |
failed for bardzo |
no alt for |
failed for uzgodnioną |
no alt for |
